In [ ]:
from yugiquery import *

init_notebook_mode(all_interactive=True)

header("My Decks")

---

# Data loading

In [ ]:
_ = git.ensure_repo()

## Read decks

In [ ]:
# Timestamp
timestamp = arrow.utcnow()

In [ ]:
# Load decks from YDK and decklist files
deck_df = pd.concat([get_ydk(), get_decklists()], ignore_index=True)

In [ ]:
# Process the deck data frame
deck_df = find_cards(deck_df, card_data=True, set_data=True)

## Changelog

In [ ]:
# Get latest file if exist
previous_df, previous_ts = load_latest_data("deck")

if previous_df is not None:
    previous_df = previous_df.astype(deck_df[previous_df.columns.intersection(deck_df.columns)].dtypes.to_dict())

In [ ]:
if previous_df is None:
    deck_changelog = None
    print("Skipped")
else:
    deck_changelog = generate_changelog(previous_df, deck_df, col="Name")
    if not deck_changelog.empty:
        display(deck_changelog)
        deck_changelog.to_csv(
            dirs.DATA
            / make_filename(
                report="deck",
                timestamp=timestamp,
                previous_timestamp=previous_ts,
            ),
            index=True,
        )
        print("Changelog saved")

In [ ]:
if deck_changelog is not None and deck_changelog.empty:
    print("No changes. New data not saved")
else:
    deck_df.to_csv(
        dirs.DATA / make_filename(report="deck", timestamp=timestamp),
        index=False,
    )
    print("Data saved")

# Data visualization

In [ ]:
deck_df

## Check collection

In [ ]:
# Merge the collection and deck data frames
collection_df = get_collection()
if collection_df is not None:
    collection_df = assign_deck(collection_df, deck_df=deck_df, return_collection=False)

In [ ]:
collection_df

## Deck composition

In [ ]:
_ = plot.deck_composition(deck_df)
plt.show()

## Attributes

In [ ]:
attr_colors = [plot.colors_dict[attr] for attr in sorted(deck_df["Attribute"].dropna().unique())]
_ = plot.deck_distribution(
    deck_df,
    "Attribute",
    colors=attr_colors,
    hatch=["", "X", "+"],
)
plt.show()

## Secondary type

In [ ]:
_ = plot.deck_distribution(deck_df.explode("Secondary type"), "Secondary type")
plt.show()

## Monster type

In [ ]:
fig = plot.deck_distribution(
    deck_df,
    "Monster type",
)
plt.show()
fig.savefig("temp.pdf")

## Properties

In [ ]:
prop_colors = [
    plot.colors_dict["Trap Card"] if "Trap" in prop else plot.colors_dict["Spell Card"]
    for prop in sorted(deck_df["Property"].dropna().unique())
]
_ = plot.deck_distribution(
    deck_df,
    "Property",
    colors=prop_colors,
    hatch=["", "X", "+"],
)
plt.show()

## TCG & OCG Status 

In [ ]:
_ = plot.deck_distribution(deck_df, "TCG status")
plt.show()

In [ ]:
_ = plot.deck_distribution(deck_df, "OCG status")
plt.show()

## Archetype & Series

In [ ]:
_ = plot.deck_distribution(deck_df.explode("Archseries"), "Archseries")
plt.show()

## ATK & DEF distribution

In [ ]:
_ = plot.deck_stem(deck_df=deck_df, y1="ATK", y2="DEF")
plt.show()

## Level & Rank distribution

In [ ]:
level = (
    deck_df[["Deck", "Section", "Level/Rank", "Count"]][deck_df["Primary type"] != "Xyz Monster"]
    .dropna()
    .rename(columns={"Level/Rank": "Level"})
)
rank = (
    deck_df[["Deck", "Section", "Level/Rank", "Count"]][deck_df["Primary type"] == "Xyz Monster"]
    .dropna()
    .rename(columns={"Level/Rank": "Rank"})
)
_ = plot.deck_stem(deck_df=pd.concat([level, rank]), y1="Level", y2="Rank")
plt.show()

## Link distribution

In [ ]:
_ = plot.deck_stem(deck_df=deck_df[deck_df["Section"] != "Main"], y1="Link")
plt.show()

## Pendulum scale distribution

In [ ]:
if deck_df["Pendulum Scale"].dropna().nunique() > 0:
    _ = plot.deck_stem(deck_df=deck_df, y1="Pendulum Scale")

# Epilogue

In [ ]:
# benchmark(report='Deck',timestamp=timestamp)

In [ ]:
footer(timestamp)

## HTML export

In [ ]:
# Save notebook on disck before generating HTML report
save_notebook()

In [ ]:
# export_notebook(dirs.NOTEBOOKS.user / "Deck.ipynb")

## Git

In [ ]:
# git.commit("*[Dd]eck*", f"Deck update - {timestamp.isoformat()}")